### AlexNet
- LeNet(그림1 좌측)은 저용량 이미지 처리에 적합했으나 대용량 이미지 처리에는 역부족이였다. 이를 해결하고자 AlexNet(그림1 우측)이 등장했다.
- Activation 함수로 ReLU 함수를 처음으로 적용한 네트워크이다.
- MaxPooling을 적용했으며, 과적합을 개선하기 위해서 Drop out Layer와 Weight Decay 기법을 적용하였다.
- Data Augmentation도 적용하여 많은 연구진들이 AlexNet의 구조를 연구하기 시작했다.
- 비교적 큰 사이즈인 11x11, 5x5 사이즈의 Kernel을 적용하였으며, 이후 3x3 Kernel을 3번 이어서 적용하였다.
- 많은 feature 정보를 추출하고자 할 때, 초기 Feature map(합성곱 연산을 통해 나온 결과)에 큰 사이즈의 Kernel을 적용하는 것이 효율적이라 판단하였다.
- 하지만 weight parameter 개수가 많아져서 CPU의 컴퓨팅 연산량이 크게 증가 하였고, 이를 극복하기 위하여 병렬 GPU를 활용할 수 있도록 하였다.
<div style="display: flex" style="margin-left: 0">
    <div>
        <img src="./images/Le_Net_Alex_Net.png" style="margin-left: -20px">
    </div>
    <div>
        <img src="./images/Alex_Net.png" width="550" style="margin-left: 20px; margin-top: 80px">
    </div>
</div>

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras import regularizers

# n_classes: 분류 개수
def alexnet(input_shape=(224, 224, 3), n_classes=10, kernel_regular=None):
    input_tensor = Input(shape=input_shape)
    
    x = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid')(input_tensor)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=3, strides=2)(x)


    x = Conv2D(filters=256, kernel_size=(5,5), strides=1, padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=3, strides=2)(x)


    # 3x3 Conv 2번 연속 적용
    x = Conv2D(filters=384, kernel_size=(3,3), strides=1, padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=384, kernel_size=(3,3), strides=1, padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    # 마지막 3x3 Conv에서 filters 수를 줄이고 maxpooling을 적용
    x = Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=3, strides=2)(x)

#     FC 연결
    x = Flatten()(x)

    # Dropout 적용. 
    x = Dense(units = 4096, activation = 'relu')(x)
    x = Dropout(0.5)(x)

    x = Dense(units = 4096, activation = 'relu')(x)
    x = Dropout(0.5)(x)

    output = Dense(units = n_classes, activation = 'softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    
    return model

In [2]:
model = alexnet(input_shape=(224, 224, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 batch_normalization (Batch  (None, 54, 54, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656